# 10.3 실습: 의미 검색 구현하기
- Semantic Search
- 단순히 키워드 매칭을 통한 검색이 아니라, 밀집 임베딩을 이용해 문장이나 문서의 의미를 고려한 검색을 수행하는 것

## 10.3.1 의미검색 구현하기

In [ ]:
# 예제 10.8 실습에 사용할 모델과 데이터셋 불러오기

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')
klue_mrc_dataset

In [ ]:
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
sentence_model

In [ ]:
# 예제 10.9 실습 데이터에서 1,000개만 선택하고 문장 임베딩으로 변환

train_data_set = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)['train']
train_data_set

In [ ]:
embeddings = sentence_model.encode(list(train_data_set['context']))
embeddings.shape

In [ ]:
# 예제 10.10 KNN 검색 인덱스를 생성하고 문장 임베딩 저장

import faiss

# 인덱스 만들기
index = faiss.IndexFlatL2(embeddings.shape[1])

# 인덱스에 임베딩 저장하기
index.add(embeddings)
index

In [ ]:
# 예제 10.11 의미 검색의 장점

query = '이번 연도에는 언제 비가 많이 올까?'
query_embeddings = sentence_model.encode([query])
distances, indices = index.search(query_embeddings, 3)  # query_embeddings와 가장 가까운 3개의 문장 검색

indices

In [ ]:
for idx in indices[0]:
    print(idx, train_data_set['context'][int(idx)], '\n-------------------------------')

In [ ]:
# 예제 10.12 의미 검색의 한계

query = klue_mrc_dataset[3]['question']  # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
query_embedding = sentence_model.encode([query])
distances, indices = index.search(query_embedding, 3)

for idx in indices[0]:
    print(klue_mrc_dataset['context'][int(idx)][:50])

## 10.3.2 라마인덱스에서 Sentence-Transformers 모델 사용하기


In [ ]:
# 예제 10.13 라마인덱스에서 Sentence-Transformers 임베딩 모델 활용

from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core import Document, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = None  # LLM을 사용하지 않으므로 None으로 설정
Settings.embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 로컬 모델 활용하기
# service_context = ServiceContext.from_defaults(embed_model="local")

text_list = klue_mrc_dataset[:100]['context']
documents = [Document(text=t) for t in text_list]

index_llama = VectorStoreIndex.from_documents(documents)
index_llama